## Problem 1: Creating trim trajectory calculator
A trim state and input produce constant dynamic outputs. One way to produce the trim states and inputs is to solve an optimization problem that results in the desired trim state derivative. Consider the objective function defined as

$$
    \min_{x_{trim}, \delta_{trim}} \bigl( f(x_{trim}, \delta_{trim}) - f_d \bigr)^T Q \bigl( f(x_{trim}, \delta_{trim}) - f_d \bigr)
$$
where $f(x_{trim}, \delta_{trim})$ denotes the dynamics given the trim state and input, $f_d$ denotes the desired dynamics, and $Q \succeq 0$ is a diagonal matrix with weights along the diagonal. If possible, the optimization will produce trim states and inputs such that $f = f_d$. 

Implement the following functions in `chap5\trim.py` using the cost and constraints discussed in lecture and described in the function docstrings.
* `velocity_constraint`
* `velocity_constraint_partial`
* `variable_bounds`
* `trim_objective_fun`

In [ ]:
# Note that this cell can be run separately to initialize for other cell blocks
import numpy as np
from mav_sim.chap3.mav_dynamics import DynamicState
from mav_sim.chap4.run_sim import run_sim
from mav_sim.message_types.msg_sim_params import MsgSimParams
from mav_sim.message_types.msg_delta import MsgDelta
from mav_sim.tools.display_figures import display_data_view, display_mav_view
from mav_sim.chap2.mav_viewer import MavViewer
from mav_sim.chap3.data_viewer import DataViewer
from mav_sim.chap5.trim import compute_trim
from IPython.display import display # Used to display variables nicely in Jupyter
from mav_sim.chap3.mav_dynamics import DynamicState, derivatives
from mav_sim.chap4.mav_dynamics import forces_moments, update_velocity_data
from mav_sim.tools.signals import Signals

# The viewers need to be initialized once due to restart issues with qtgraph
if 'mav_view' not in globals():
    print("Initializing mav_view")
    global mav_view
    mav_view = MavViewer()  # initialize the mav viewer
if 'data_view' not in globals():
    print("Initializing data_view")
    global data_view
    data_view = DataViewer()  # initialize view of data plots

# Initialize state values
sim_params = MsgSimParams(end_time=40., video_name="chap5.avi") 
state = DynamicState()

# Functions used below
def run_sim_and_display(delta_fnc, init_state):
    global mav_view
    global data_view
    data_view.reset(sim_params.start_time)
    (mav_view, data_view) = run_sim(sim_params, delta_fnc, DynamicState(init_state), mav_view, data_view)
    display_data_view(data_view)
    display_mav_view(mav_view)

## Problem 2 - Calculate Trim
Do the following:
1. Compute a trim state and input for $V_a = 25 \frac{m}{s}$, $\gamma = 0$, and $R=\infty$
2. Show that the resulting dynamics are close to the desired dynamics
3. Simulate the state and show that the trim values are achieved for a period before numerical errors enter the system

Keep in mind the following when comparing results:
* $\dot{p_n}$ and $\dot{p_e}$ are not important
* Your trim state dynamics will not be exact, but they should be close

In [ ]:
# Create the trim state
Va_trim = 25.
gamma_trim = 0.
R = np.inf
trim_state, trim_input = compute_trim(state.convert_to_numpy(), Va_trim, gamma_trim, R)

# Display the trim state and input
print('trim_state = ')
display(trim_state)
print('trim input = ')
trim_input.print()

# Calculate the desired state dynamics
desired_trim_state_dot = np.array([[0., 0., -Va_trim*np.sin(gamma_trim), 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]).T

# Calculate the actual state dynamics
(Va, alpha, beta, _) = update_velocity_data(trim_state)
forces_moments_vec = forces_moments(trim_state, trim_input, Va, beta, alpha)
f = derivatives(trim_state, forces_moments_vec) 

# Display the difference
f_diff = f - desired_trim_state_dot
print("Difference between actual and desired (Note that pn and pe are not important):")
display(f_diff)

# Create a passthrough function for the trim input
pass_delta2 = lambda sim_time: trim_input
run_sim_and_display(pass_delta2, trim_state)

## Problem 3 - Trim Calculations
Repeat problem 2 with $V_a = 25 \frac{m}{s}$, $\gamma = 5$ degrees, and $R=\infty$


In [ ]:
# Create the trim state
Va_trim = 25.
gamma_trim = 5*np.pi/180.
trim_state, trim_input = compute_trim(state.convert_to_numpy(), Va_trim, gamma_trim)

# Display the trim state and input
print('trim_state = ')
display(trim_state)
print('trim input = ')
trim_input.print()

# Calculate the desired state dynamics
desired_trim_state_dot = np.array([[0., 0., -Va_trim*np.sin(gamma_trim), 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]).T

# Calculate the actual state dynamics
(Va, alpha, beta, _) = update_velocity_data(trim_state)
forces_moments_vec = forces_moments(trim_state, trim_input, Va, beta, alpha)
f = derivatives(trim_state, forces_moments_vec) 

# Display the difference
f_diff = f - desired_trim_state_dot
print("Difference between actual and desired (Note that pn and pe are not important):")
display(f_diff)

# Create a passthrough function for the trim input
pass_delta2 = lambda sim_time: trim_input
run_sim_and_display(pass_delta2, trim_state)

## Problem 4 - Trim Calculations
Repeat problem 2 with $V_a = 30 \frac{m}{s}$, $\gamma = 3$ degrees, and $R=800 m$ 

In [ ]:
from mav_sim.chap3.mav_dynamics_euler import derivatives_euler, quat_state_to_euler_state

# Create the trim state
Va_trim = 30.
gamma_trim = 3*np.pi/180.
R = 800.
trim_state, trim_input = compute_trim(state.convert_to_numpy(), Va_trim, gamma_trim, R)

# Display the trim state and input
print('trim_state = ')
display(trim_state)
print('trim input = ')
trim_input.print()

# Calculate the desired state dynamics
desired_trim_state_dot = np.array([[0., 0., -Va_trim*np.sin(gamma_trim), 0., 0., 0., 0., 0., Va_trim/R*np.cos(gamma_trim), 0., 0., 0.]]).T

# Calculate the actual state dynamics
(Va, alpha, beta, _) = update_velocity_data(trim_state)
forces_moments_vec = forces_moments(trim_state, trim_input, Va, beta, alpha)
trim_state_euler = quat_state_to_euler_state(trim_state)
f = derivatives_euler(trim_state_euler, forces_moments_vec) 

# Display the difference
f_diff = f - desired_trim_state_dot
print("Difference between actual and desired (Note that pn and pe are not important):")
display(f_diff)

# Create a passthrough function for the trim input
pass_delta2 = lambda sim_time: trim_input
run_sim_and_display(pass_delta2, trim_state)

## Problem 5 - Evaluate Eigenvalues of Longitudinal System
The `compute_ss_model(...)` function inside `chap5\compute_models.py` provides a numerical approximation for the models described in (5.44) and (5.51). 

For the trim trajectory corresponding to $V_a = 25 \frac{m}{s}$ and $\gamma = 0$, do the following:
* Calculate the eigenvalues of $A_{lon}$ and $A_{lat}$
* Answer the questions below

### Question: Which eigenvalue(s) correspond to the short-period mode?
Complex conjugate pair of eigenvalues of $A_{lon}$ with with the larger $\omega_n$ correspond to the short-period mode.

$\lambda = -4.87859821 \pm 9.86957041j$

### Question: Which eigenvalue(s) correspond to the phugoid mode?
Complex conjugate pair of eigenvalues of $A_{lon}$ with with the smaller $\omega_n$ correspond to the short-period mode.

$\lambda = -0.10413332 \pm 0.48908574j$

### Question: Which eigenvalue(s) corresponds to the spiral-divergence mode?
Corresponds to the real eigenvalue in the right half plane

$\lambda = 8.93941757e-02$

### Question: Which eigenvalue(s) corresponds to the roll mode?
Corresponds to the real eigenvalue in the left half plane

$\lambda = -2.24411598e+01$

### Question: Which eigenvalue(s) corresponds to the dutch-roll mode?
Correspond to the complex eigenvalues

$\lambda = -1.14075619e \pm 00+4.65506181j $


In [ ]:
from mav_sim.chap5.compute_models import compute_ss_model

# Compute the trim state and input
Va_trim = 25.
gamma_trim = 0.
trim_state, trim_input = compute_trim(state.convert_to_numpy(), Va_trim, gamma_trim)

# Compute A_lon and A_lat
A_lon, B_lon, A_lat, B_lat = compute_ss_model(trim_state, trim_input)

# Compute the eigenvalues of A_lon for calculating short-period and phugoid modes
eig_A_lon, _ = np.linalg.eig(A_lon)
print("Eigenvalues of A_lon")
display(eig_A_lon)
print("Natural frequencies of A_lon")
w_n_lon = np.abs(eig_A_lon)
display(w_n_lon)

# Compute the eigenvalues of A_lat for calculating the spiral-divergence mode, roll mode, and dutch-roll mode
eig_A_lat, _ = np.linalg.eig(A_lat)
print("Eigenvalues of A_lat")
display(eig_A_lat)


## Problem 6 - Phugoid mode
For the trim trajectory corresponding to $V_a = 25 \frac{m}{s}$ and $\gamma = 0$, use a doublet to excite the phugoid mode. Simulate the response. (Note that this problem is provided for you)

In [ ]:
# Create the trim state
Va_trim = 25.
gamma_trim = 0.
trim_state, trim_input = compute_trim(state.convert_to_numpy(), Va_trim, gamma_trim)

# Create an input signal
input_signal = Signals(amplitude=0.3,
                    duration=0.3,
                    start_time=5.0)

# Create a function for exciting the phugoid mode
def excite_phugoid(sim_time: float):
    # copy the trim command
    delta_cmd = MsgDelta()
    delta_cmd.copy(trim_input)

    # Excite the phugoid mode
    delta_cmd.elevator += input_signal.doublet(sim_time)
    return delta_cmd

# Run the command
run_sim_and_display(excite_phugoid, trim_state)

## Problem 7 - Roll and spiral divergence modes
For the trim trajectory corresponding to $V_a = 25 \frac{m}{s}$ and $\gamma = 0$, use a doublet to excite the roll and spiral divergence modes.

In [ ]:
# Create the trim state
Va_trim = 25.
gamma_trim = 0.
trim_state, trim_input = compute_trim(state.convert_to_numpy(), Va_trim, gamma_trim)

# Create an input signal
input_signal = Signals(amplitude=0.3,
                    duration=0.3,
                    start_time=5.0)

# Create a function for exciting the phugoid mode
def excite_roll(sim_time: float):
    # copy the trim command
    delta_cmd = MsgDelta()
    delta_cmd.copy(trim_input)

    # Excite the phugoid mode
    delta_cmd.aileron += input_signal.doublet(sim_time)
    return delta_cmd

# Run the command
run_sim_and_display(excite_roll, trim_state)

## Problem 8 - Dutch roll mode
For the trim trajectory corresponding to $V_a = 25 \frac{m}{s}$ and $\gamma = 0$, use a doublet to excite the dutch roll mode.

In [ ]:
# Create the trim state
Va_trim = 25.
gamma_trim = 0.
trim_state, trim_input = compute_trim(state.convert_to_numpy(), Va_trim, gamma_trim)

# Create an input signal
input_signal = Signals(amplitude=0.3,
                    duration=0.3,
                    start_time=5.0)

# Create a function for exciting the phugoid mode
def excite_dutch_roll(sim_time: float):
    # copy the trim command
    delta_cmd = MsgDelta()
    delta_cmd.copy(trim_input)

    # Excite the phugoid mode
    delta_cmd.rudder += input_signal.doublet(sim_time)
    return delta_cmd

# Run the command
run_sim_and_display(excite_dutch_roll, trim_state)

## Static analysis
Run the static code analysis (you must have zero static code analysis errors to get credit). You may not modify the static code analysis configuration files.

### ISORT
Run Isort:
```
python -m isort mav_sim book_assignments
```

Terminal output (should be nothing):

### MyPy
Run MyPy
```
python -m mypy mav_sim/chap2/ mav_sim/chap3/ mav_sim/chap4/ mav_sim/chap5/ book_assignments
```

Terminal output (should indicate no error):
```
Success: no issues found in 31 source files
```

### Pylint
Run Pylint
```
python -m pylint --jobs 0 --rcfile .pylintrc mav_sim/chap2/ mav_sim/chap3/ mav_sim/chap4/ mav_sim/chap5/ book_assignments/
```

Terminal output (should indicate `10/10`)
```
------------------------------------
Your code has been rated at 10.00/10
```

## Simple code checking
The following code does not need to change. It should just be used as a sanity check so that you know the code is implemented properly. The output should not have any lines reading `Failed test!`

Note that due to the random nature of the test, you may get a warning stating
```
RuntimeWarning: invalid value encountered in sqrt
  omega_p = (-b + np.sqrt(b**2 - 4*a*c)) / (2.*a)
```
Just ignore that warning.

In [ ]:
from mav_sim.unit_tests.ch5_dynamics_test import run_auto_tests, VelocityConstraintTest, VelocityConstraintPartialTest, VariableBoundsTest, TrimObjectiveFunTest
run_auto_tests()